In [1]:
import random
from collections import defaultdict
from pathlib import Path
from typing import Dict, List

import fire
import numpy as np
from PIL import Image
from sahi.utils.coco import Coco, CocoAnnotation, CocoCategory, CocoImage
from sahi.utils.file import load_json, save_json
from tqdm import tqdm

/vol/bitbucket/jrb21/anaconda3/envs/sodb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_images_dir = "/vol/bitbucket/jrb21/project/xView/data/train/images"
train_geojson_path = "/vol/bitbucket/jrb21/project/xView/data/xView_train.geojson"
output_dir = "/vol/bitbucket/jrb21/project/xView/data/coco"
train_split_rate=0.75
category_id_remapping=None

In [3]:
# init vars
category_id_to_name = {}
with open("xview/xview_class_labels.txt", encoding="utf8") as f:
    lines = f.readlines()
for line in lines:
    category_id = line.split(":")[0]
    category_name = line.split(":")[1].replace("\n", "")
    category_id_to_name[category_id] = category_name
    
    
category_id_to_name

{'11': 'Fixed-wing Aircraft',
 '12': 'Small Aircraft',
 '13': 'Cargo Plane',
 '15': 'Helicopter',
 '17': 'Passenger Vehicle',
 '18': 'Small Car',
 '19': 'Bus',
 '20': 'Pickup Truck',
 '21': 'Utility Truck',
 '23': 'Truck',
 '24': 'Cargo Truck',
 '25': 'Truck w/Box',
 '26': 'Truck Tractor',
 '27': 'Trailer',
 '28': 'Truck w/Flatbed',
 '29': 'Truck w/Liquid',
 '32': 'Crane Truck',
 '33': 'Railway Vehicle',
 '34': 'Passenger Car',
 '35': 'Cargo Car',
 '36': 'Flat Car',
 '37': 'Tank car',
 '38': 'Locomotive',
 '40': 'Maritime Vessel',
 '41': 'Motorboat',
 '42': 'Sailboat',
 '44': 'Tugboat',
 '45': 'Barge',
 '47': 'Fishing Vessel',
 '49': 'Ferry',
 '50': 'Yacht',
 '51': 'Container Ship',
 '52': 'Oil Tanker',
 '53': 'Engineering Vehicle',
 '54': 'Tower crane',
 '55': 'Container Crane',
 '56': 'Reach Stacker',
 '57': 'Straddle Carrier',
 '59': 'Mobile Crane',
 '60': 'Dump Truck',
 '61': 'Haul Truck',
 '62': 'Scraper/Tractor',
 '63': 'Front loader/Bulldozer',
 '64': 'Excavator',
 '65': 'Cement

In [4]:
if category_id_remapping is None:
    category_id_remapping = load_json("xview/category_id_mapping.json")
category_id_remapping

{'11': '0',
 '12': '1',
 '13': '2',
 '15': '3',
 '17': '4',
 '18': '5',
 '19': '6',
 '20': '7',
 '21': '8',
 '23': '9',
 '24': '10',
 '25': '11',
 '26': '12',
 '27': '13',
 '28': '14',
 '29': '15',
 '32': '16',
 '33': '17',
 '34': '18',
 '35': '19',
 '36': '20',
 '37': '21',
 '38': '22',
 '40': '23',
 '41': '24',
 '42': '25',
 '44': '26',
 '45': '27',
 '47': '28',
 '49': '29',
 '50': '30',
 '51': '31',
 '52': '32',
 '53': '33',
 '54': '34',
 '55': '35',
 '56': '36',
 '57': '37',
 '59': '38',
 '60': '39',
 '61': '40',
 '62': '41',
 '63': '42',
 '64': '43',
 '65': '44',
 '66': '45',
 '71': '46',
 '72': '47',
 '73': '48',
 '74': '49',
 '76': '50',
 '77': '51',
 '79': '52',
 '83': '53',
 '84': '54',
 '86': '55',
 '89': '56',
 '91': '57',
 '93': '58',
 '94': '59'}

In [5]:
# init coco object
coco = Coco()
# append categories
for category_id, category_name in category_id_to_name.items():
    if category_id in category_id_remapping.keys():
        remapped_category_id = category_id_remapping[category_id]
        coco.add_category(
            CocoCategory(id=int(remapped_category_id), name=category_name)
        )

In [47]:
def get_labels(fname):
    """
    Gets label data from a geojson label file
    Args:
        fname: file path to an xView geojson label file
    Output:
        Returns three arrays: coords, chips, and classes corresponding to the
            coordinates, file-names, and classes for each ground truth.
    Modified from https://github.com/DIUx-xView.
    """
    data = load_json(fname)

    coords = np.zeros((len(data["features"]), 4))
    chips = np.zeros((len(data["features"])), dtype="object")
    classes = np.zeros((len(data["features"])))
    image_name_to_annotation_ind = defaultdict(list)

    for i in tqdm(range(len(data["features"])), "Parsing xView data"):
        if data["features"][i]["properties"]["bounds_imcoords"] != []:
            b_id = data["features"][i]["properties"]["image_id"]
            # https://github.com/DIUx-xView/xView1_baseline/issues/3
            if b_id == "1395.tif":
                continue
            val = np.array(
                [
                    int(num)
                    for num in data["features"][i]["properties"][
                        "bounds_imcoords"
                    ].split(",")
                ]
            )
            chips[i] = b_id
            classes[i] = data["features"][i]["properties"]["type_id"]

            image_name_to_annotation_ind[b_id].append(i)

            if val.shape[0] != 4:
                print("Issues at %d!" % i)
            else:
                coords[i] = val
        else:
            chips[i] = "None"

    return coords, chips, classes, image_name_to_annotation_ind

In [48]:
    # parse xview data
    coords, chips, classes, image_name_to_annotation_ind = get_labels(
        train_geojson_path)

Parsing xView data: 100%|███████████████████████████████████████████████████| 601937/601937 [00:01<00:00, 334990.31it/s]


In [53]:
image_name_to_annotation_ind["2355.tif"]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [54]:
image_name_to_annotation_ind.keys()

dict_keys(['2355.tif', '2356.tif', '2370.tif', '2371.tif', '2384.tif', '2386.tif', '2387.tif', '2398.tif', '2399.tif', '2268.tif', '2270.tif', '2278.tif', '2279.tif', '2281.tif', '2292.tif', '2293.tif', '2294.tif', '2301.tif', '2303.tif', '2305.tif', '2306.tif', '2308.tif', '2309.tif', '2313.tif', '2314.tif', '2322.tif', '2334.tif', '2408.tif', '2421.tif', '2423.tif', '1559.tif', '1562.tif', '1577.tif', '1579.tif', '1580.tif', '1581.tif', '1600.tif', '1602.tif', '1604.tif', '1624.tif', '1625.tif', '1647.tif', '1649.tif', '1651.tif', '72.tif', '118.tif', '136.tif', '1449.tif', '1451.tif', '1457.tif', '1460.tif', '1462.tif', '1466.tif', '1468.tif', '1473.tif', '2538.tif', '2541.tif', '2545.tif', '2547.tif', '2555.tif', '2557.tif', '2559.tif', '2560.tif', '2562.tif', '2564.tif', '282.tif', '283.tif', '285.tif', '287.tif', '289.tif', '291.tif', '293.tif', '294.tif', '295.tif', '296.tif', '302.tif', '303.tif', '307.tif', '309.tif', '310.tif', '311.tif', '315.tif', '317.tif', '320.tif', '321

In [50]:
def get_ordered_image_name_list(image_name_to_annotation_ind: Dict):
    image_name_list: List[str] = list(image_name_to_annotation_ind.keys())

    def get_image_ind(image_name: str):
        return int(image_name.split(".")[0])

    image_name_list.sort(key=get_image_ind)

    return image_name_list

In [55]:
image_name_list = get_ordered_image_name_list(image_name_to_annotation_ind)

In [56]:
image_name_list

['5.tif',
 '8.tif',
 '10.tif',
 '18.tif',
 '20.tif',
 '24.tif',
 '31.tif',
 '33.tif',
 '38.tif',
 '40.tif',
 '41.tif',
 '42.tif',
 '43.tif',
 '46.tif',
 '47.tif',
 '53.tif',
 '69.tif',
 '72.tif',
 '73.tif',
 '74.tif',
 '75.tif',
 '79.tif',
 '80.tif',
 '83.tif',
 '84.tif',
 '86.tif',
 '87.tif',
 '88.tif',
 '89.tif',
 '90.tif',
 '91.tif',
 '92.tif',
 '94.tif',
 '95.tif',
 '97.tif',
 '99.tif',
 '100.tif',
 '102.tif',
 '104.tif',
 '105.tif',
 '106.tif',
 '107.tif',
 '109.tif',
 '110.tif',
 '111.tif',
 '112.tif',
 '118.tif',
 '124.tif',
 '125.tif',
 '126.tif',
 '128.tif',
 '129.tif',
 '130.tif',
 '131.tif',
 '136.tif',
 '140.tif',
 '142.tif',
 '144.tif',
 '145.tif',
 '149.tif',
 '157.tif',
 '158.tif',
 '159.tif',
 '163.tif',
 '180.tif',
 '181.tif',
 '193.tif',
 '203.tif',
 '216.tif',
 '217.tif',
 '223.tif',
 '237.tif',
 '238.tif',
 '252.tif',
 '254.tif',
 '282.tif',
 '283.tif',
 '285.tif',
 '287.tif',
 '289.tif',
 '291.tif',
 '293.tif',
 '294.tif',
 '295.tif',
 '296.tif',
 '302.tif',
 '303.

In [ ]:
# convert xView data to COCO format
for image_name in tqdm(image_name_list, "Converting xView data into COCO format"):
    # create coco image object
    width, height = Image.open(Path(train_images_dir) / image_name).size
    coco_image = CocoImage(file_name=image_name, height=height, width=width)

    annotation_ind_list = image_name_to_annotation_ind[image_name]

    # iterate over image annotations
    for annotation_ind in annotation_ind_list:
        bbox = coords[annotation_ind].tolist()
        category_id = str(int(classes[annotation_ind].item()))
        coco_bbox = [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]]
        if category_id in category_id_remapping.keys():
            category_name = category_id_to_name[category_id]
            remapped_category_id = category_id_remapping[category_id]
        else:
            continue
        # create coco annotation and append it to coco image
        coco_annotation = CocoAnnotation(
            bbox=coco_bbox,
            category_id=int(remapped_category_id),
            category_name=category_name,
        )
        if coco_annotation.area > 0:
            coco_image.add_annotation(coco_annotation)
    coco.add_image(coco_image)

In [58]:
category_id_remapping